**Setup the project**
1. Create a folder for the project
2. Create the virtual environment for this project using ```python -m venv .venv``` (make sure the terminal is inside the projects folder)
3. Activate the environment ```.\.venv\Scripts\activate```. 
4. You should see (venv) in the beginning of the prompt. If you cannot see such (because of oh-my-posh for example), try with this command ```python -c "import sys; print(sys.executable)"``` if the result is pointing to yourpoject\.venv\scripts\python.exe, then it is activated (like C:\Projects\Python\FirstProject\.venv\Scripts\python.exe)

**Add dependencies**
1. Add kernel ```pip install ipykernel```
2. Set the env variables:  ```pip install requests python-dotenv rich```
3. Add Pandas and NumPy: ```pip install numpy pandas matplotlib```
4. Install torchvision by running in the terminal of the project (while vinv is activated) ```pip install torch torchvision```

**Setup VS Code (if needed)**
1. If you want to have nicer python environment, add the following to VisualStudio Code (Ctrl+Shift+P -> Preferences: Open User Settings (JSON))
```json
    "editor.formatOnSave": true,

    "[python]": {
        "editor.defaultFormatter": "ms-python.black-formatter",
        "editor.formatOnType": true
    },

    "python.analysis.typeCheckingMode": "basic",
    "python.testing.pytestEnabled": true,

    "jupyter.notebookFileRoot": "${workspaceFolder}"
```
2. If you want python linter, install it using ```pip install ruff```
3.  ..and add support in VS Code Preferencs (as in point 8)
```json
    "ruff.lint.enable": true,
    "ruff.format.enable": true,
```
4.  Set the notebook kernel with the python.exe from the current .venv by choosing it manually 

Then you should be able to run the script from above 

In [ ]:
import torch                                            ## Load PyTorch
from torchvision import datasets, transforms            ## Load torchvision - computer vision library (MNIST is part of the dataset package)
from torch.utils.data import DataLoader                 ## Enable the dataloader to the dataset

transform = transforms.ToTensor()                       ## transform the image from MNIST to tensors

## create the datasets.
# download them into data/ folder;
# train=true/false indicates which is for training and which for testing
# download=true indicate to download it if it is not already downloaded
# transform=transform - apply transform to each image
train = datasets.MNIST("data", train=True, download=True, transform=transform)
test  = datasets.MNIST("data", train=False, download=True, transform=transform)

## Loads the data into dataloaders on batches (batch_size=64) in shuffled order (per epoch only for the train dataset )
train_loader = DataLoader(train, batch_size=64, shuffle=True)
test_loader  = DataLoader(test, batch_size=1000) ## for the test the shuffle is not needed. the batch also could be bigger

## create the linear regression (y=Wx+b)
## 28*28 - each MINST image is 28*28 pixels - this is the in_features
## we need to find the number which the image represent (0,1,2,3,4,5,6,7,8,9) = 10. this is the out_features
model = torch.nn.Linear(28*28, 10) ##

## Set the loss
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)


Train the model

In [11]:
epochs = 3

for epoch in range(epochs):
    total_loss = 0

    for images, labels in train_loader:
        # 1) Flatten: от 1×28×28 → 784
        X = images.view(images.size(0), -1)

        # 2) Forward
        logits = model(X)

        # 3) Loss
        loss = loss_fn(logits, labels)

        # 4) Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, loss = {total_loss:.3f}")


Epoch 1, loss = 448.714
Epoch 2, loss = 316.287
Epoch 3, loss = 294.806


Test it

In [12]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        X = images.view(images.size(0), -1)
        logits = model(X)
        predictions = torch.argmax(logits, dim=1)

        correct += (predictions == labels).sum().item()
        total += labels.size(0)

print("Accuracy:", correct / total)


Accuracy: 0.9181
